![logo](./img/LogoLine_horizon_C3S.png)

<br>

# PREPARE FILES AND RUN PERSIST and INCA FOR HINDCAST PERIOD

### About

1. Create a .bat file to run PERSIST
2. Run PERSIST for all hindcast files
3. Correct HER outputs from PERSIST
4. Extract SOC time series from INCA forced with ERA5 to reset initial conditions of SOC in hindcast runs (when required)
5. Run INCA and process outputs
6. Process INCA outputs from ERA5  (when required)

### Install packages

In [1]:
# Miscellaneous operating system interfaces
import os

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Copy files to another path directory
import shutil

#Time
import time

#to run models
import subprocess

In [3]:
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/'
os.chdir(path) #always run this line before runing a new test

# 1. Create .bat file with all hindcast .par and .dat files 

In [3]:
#Files extracted from "hindcast_bc_persist_input" folder for .dat files
#The best optimized .par files (october 2024) were used to create .par files (location: hindcast_run_model) 

# Open the .bat file in write mode
with open(f'{path}hindcast_persist_files/bat_persist.bat', 'w') as bat_file:
    # Loop through members, years, and months
    for member in range(0, 25):  # member from 0 to 24
        for y in range(0, 24):  # year from 0 to 23
            for m in range(0, 12):  # month from 0 to 11
                
                month = 1 + m
                year = 1993 + y
                start_year = year - 5  # IMPORTANT: Change the number of warmup years if required !!!!!!!!!!!!
                            
                # Update PERSiST parameters
                with open(f'{path}hindcast_bc_persist_input/hind_merged__member{member}_year{y}_month{m}.csv', 'r') as file:
                    lines_dat = file.readlines()   # take Steps number from .dat file
                    
                with open(f'{path}hindcast_run_model/persist_par.par', 'r') as file:
                    lines = file.readlines()

                lines[0] = lines_dat[0]          # update Steps number
                lines[1] = f'1/{month}/{start_year}\n' # update Date

                # Save .par files
                par_filename = f'persist_par_member{member}_year{y}_month{m}.par'
                with open(f'{path}hindcast_persist_files/{par_filename}', 'w') as file:
                    file.writelines(lines)
                
                # Remove quotes from CSV content before saving as .dat
                dat_filename = f'persist_dat_member{member}_year{y}_month{m}.dat'
                
                with open(f'{path}hindcast_bc_persist_input/hind_merged__member{member}_year{y}_month{m}.csv', 'r') as infile:
                    lines = infile.readlines()
                
                with open(f'{path}hindcast_persist_files/{dat_filename}', 'w') as outfile:
                    for line in lines:
                        # Remove quotes from the line
                        clean_line = line.replace('"', '')
                        outfile.write(clean_line)
                
                # Write the command to the .bat file
                cmd = f'persist_cmd_0.exe -par {par_filename} -dat {dat_filename} -inca cmd_member{member}_year{y}_month{m} -out output_member{member}_year{y}_month{m} -size all\n'
                bat_file.write(cmd)


# 2. Execute the created .bat file (double click) to run PERSIST for the hindcast .bat file (bat_persist) performed in the "hindcast_persist_files" folder. The cmd version of PERSiST, "persist_cmd_0.exe", must be present in the same folder 

# 3. Recalculate HER manually for Subcatchments C1 and C2 (code developed based on formulas provided by Martyn Futter)

### 3.1 Subcatchment C1

In [ ]:
#Function to calculate HER
def calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file):

    # Rain and snow melt calculation
    rainPlusSnowmelt = [[], [], [], [], []]

    for i in range(5):
        reach_c1_found = False  # Reset the flag for each file

        with open(snowFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C1' in line:
                    reach_c1_found = True
                    continue
                if 'Snowfall,Snow melt,Snow depth,Rain' in line:
                    continue
                if 'C2' in line:
                    break
            
                columns = line.strip().split(',')
                if reach_c1_found and len(columns) == 5:
                    snow_melt = float(columns[2].strip())
                    rain = float(columns[4].strip())
                    rainPlusSnowmelt[i].append(snow_melt + rain)
                
    rainPlusSnowmelt = np.array(rainPlusSnowmelt)

    # Runoff extraction
    runOff = [[], [], [], [], []]

    for i in range(5):
        reach_c1_found = False  # Reset the flag for each file

        with open(runOffFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C1' in line:
                    reach_c1_found = True
                    continue
                if 'Runoff' in line:
                    continue
                if 'C2' in line:
                    break
            
                columns = line.strip().split(',')
                if reach_c1_found and len(columns) == 2:
                    runOff[i].append(float(columns[1].strip()))

    runOff = np.array(runOff)

    # HER calculation
    HER = np.zeros([5, runOff.shape[1]])
    percValues = [0.0609, 0.2333, 0.3533, 0.3423, 0.0102] #land use percentages for C1
    cumRunOff = np.zeros([5, runOff.shape[1]])
    cumRunOff[:, -1] = runOff[:, -1]

    for i in range(5):
        for j in np.arange(runOff.shape[1] - 2, -1, -1):
            cumRunOff[i, j] = cumRunOff[i, j + 1] + runOff[i, j] - HER[i, j + 1]
            HER[i, j] = np.min([rainPlusSnowmelt[i, j], cumRunOff[i, j]])

    HER_partial = np.zeros([5, runOff.shape[1]])
    for i in range(5):
        HER_partial[i, :] = percValues[i] * HER[i, :]

    HER_final = np.sum(HER_partial, axis=0)


    # Change the corrected HER values (HER_final) into the HER_from_persist file
    df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
    # Update the second column (index 1) with HER_final values
    df.iloc[:, 1] = HER_final  # Directly assign HER_final values to the second column
    # Write the updated DataFrame back to a CSV file
    df.to_csv(output_file, index=False, header=False, sep= ' ')  # Use index=False to avoid adding an extra index column

    return HER_final


#Main loop
for member in range(0, 25):  # member from 0 to 24
    for y in range(0, 24):    # year from 0 to 23
        for m in range(0, 12):  # month from 0 to 11
            # Define file paths for each iteration
            snow_agr = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Agriculture.csv'
            snow_broad_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Broad_leaved_Forest.csv'
            snow_conf_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Coniferous_Forest.csv'
            snow_small_veg = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Small_NoVegetation.csv'
            snow_urb = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Urban.csv'

            snowFiles = [snow_agr, snow_broad_forest, snow_conf_forest, snow_small_veg, snow_urb]

            runoff_agr = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Agriculture.csv'
            runoff_broad_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Broad_leaved_Forest.csv'
            runoff_conf_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Coniferous_Forest.csv'
            runoff_small_veg = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Small_NoVegetation.csv'
            runoff_urb = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Urban.csv'

            runOffFiles = [runoff_agr, runoff_broad_forest, runoff_conf_forest, runoff_small_veg, runoff_urb]
            
            HER_from_persist = f'{path}/hindcast_persist_files/cmd_member{member}_year{y}_month{m} C1.dat'
            output_file = f'{path}/hindcast_inca_files/inca_C1_member{member}_year{y}_month{m}.dat'

            # Call the HER calculation function with current file paths
            HER_final = calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file)

            # Optionally
            print(f'Processed member={member}, year={y}, month={m}')



### 3.2 Subcatchment C2

In [ ]:
#Function to calculate HER
def calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file):

    # Rain and snow melt calculation
    rainPlusSnowmelt = [[], [], [], [], []]

    for i in range(5):
        reach_c2_found = False  # Reset the flag for each file

        with open(snowFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C2' in line:
                    reach_c2_found = True
                    continue
                if 'Snowfall,Snow melt,Snow depth,Rain' in line:
                    continue
                
                columns = line.strip().split(',')
                
                if reach_c2_found and len(columns) == 5:
                    snow_melt = float(columns[2].strip())
                    rain = float(columns[4].strip())
                    rainPlusSnowmelt[i].append(snow_melt + rain)
                
    rainPlusSnowmelt = np.array(rainPlusSnowmelt)

    # Runoff extraction
    runOff = [[], [], [], [], []]

    for i in range(5):
        reach_c2_found = False  # Reset the flag for each file

        with open(runOffFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C2' in line:
                    reach_c2_found = True
                    continue
                if 'Runoff' in line:
                    continue
                            
                columns = line.strip().split(',')
                
                if reach_c2_found and len(columns) == 2:
                    runOff[i].append(float(columns[1].strip()))

    runOff = np.array(runOff)

    # HER calculation
    HER = np.zeros([5, runOff.shape[1]])
    percValues = [0.3097, 0.3643, 0.2372, 0.0397, 0.0491] #land use percentages for C2
    cumRunOff = np.zeros([5, runOff.shape[1]])
    cumRunOff[:, -1] = runOff[:, -1]

    for i in range(5):
        for j in np.arange(runOff.shape[1] - 2, -1, -1):
            cumRunOff[i, j] = cumRunOff[i, j + 1] + runOff[i, j] - HER[i, j + 1]
            HER[i, j] = np.min([rainPlusSnowmelt[i, j], cumRunOff[i, j]])

    HER_partial = np.zeros([5, runOff.shape[1]])
    for i in range(5):
        HER_partial[i, :] = percValues[i] * HER[i, :]

    HER_final = np.sum(HER_partial, axis=0)
    


    # Change the corrected HER values (HER_final) into the HER_from_persist file
    df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
    # Update the second column (index 1) with HER_final values
    df.iloc[:, 1] = HER_final  # Directly assign HER_final values to the second column
    # Write the updated DataFrame back to a CSV file
    df.to_csv(output_file, index=False, header=False, sep= ' ')  # Use index=False to avoid adding an extra index column

    return HER_final


#Main loop
for member in range(0, 25):  # member from 0 to 24
    for y in range(0, 24):    # year from 0 to 23
        for m in range(0, 12):  # month from 0 to 11
            # Define file paths for each iteration
            snow_agr = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Agriculture.csv'
            snow_broad_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Broad_leaved_Forest.csv'
            snow_conf_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Coniferous_Forest.csv'
            snow_small_veg = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Small_NoVegetation.csv'
            snow_urb = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_snow_Urban.csv'

            snowFiles = [snow_agr, snow_broad_forest, snow_conf_forest, snow_small_veg, snow_urb]

            runoff_agr = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Agriculture.csv'
            runoff_broad_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Broad_leaved_Forest.csv'
            runoff_conf_forest = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Coniferous_Forest.csv'
            runoff_small_veg = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Small_NoVegetation.csv'
            runoff_urb = f'{path}/hindcast_persist_files/output_member{member}_year{y}_month{m}_runoff_Urban.csv'

            runOffFiles = [runoff_agr, runoff_broad_forest, runoff_conf_forest, runoff_small_veg, runoff_urb]

            HER_from_persist = f'{path}/hindcast_persist_files/cmd_member{member}_year{y}_month{m} C2.dat'
            output_file = f'{path}/hindcast_inca_files/inca_C2_member{member}_year{y}_month{m}.dat'

            # Call the HER calculation function with current file paths
            HER_final = calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file)

            # Optionally
            #print(f'Processed member={member}, year={y}, month={m}')

# 4. Extracting SOC conditions from ERA5 (This is only required when SOC initial values forced with a new ERA5 dataset is required) 

#### Note: SOC values were extracted for C1 and C2, but only Organic and Mineral SOC values from C1 were used as initial conditions of the system (after tests using C2 and C1-C2 average)

### 4.1 Organic layer SOC C1

In [ ]:
encodings = ['latin1', 'iso-8859-1', 'utf-8']
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# List of suffixes for the files to process
file_suffixes = ['OL1', 'OL2', 'OL3', 'OL4', 'OL5', 'OL6']

# Define the output file
output_directory = f'{path}reanalysis/'
output_file = f'{output_directory}Organic_SOC_C1_10year.txt'   #We used 10-year and 30-year runs of ERA5

# Initialize a list to store the final data table
final_data = []

# Process each file
for idx, suffix in enumerate(file_suffixes):
    input_file = f'{path}reanalysis/inca_output_ERA5.{suffix}'
    
    # Try reading the file with different encodings
    for encoding in encodings:
        try:
            with open(input_file, 'r', encoding=encoding) as file:
                lines = file.readlines()
            break  # If reading is successful, break out of the loop
        except UnicodeDecodeError:
            print(f"Failed to decode {input_file} with encoding {encoding}. Trying next encoding...")

    # Initialize variables for processing
    reach_c1_found = False
    file_data = []

    # Process each line
    for line in lines:
        # Check for the start of the relevant data block
        if 'Sub-catchment 1 - 1' in line:
            reach_c1_found = True
            continue  # Skip this line and move to the next

        # Break if the end of the relevant block is found
        if 'Sub-catchment 2 - 2' in line:
            break

        # Process lines within the relevant block
        if reach_c1_found:
            # Split the line into columns
            columns = line.split()

            # Check if the line has the required number of columns
            if len(columns) >= 7:
                try:
                    # Extract the date and SOC value
                    date = columns[0]  # Date (first column)
                    Fast_SOC = float(columns[2])  # Second column (convert to float)
                    Slow_SOC = float(columns[6])  # Sixth column (convert to float)
                    SOC_value = Fast_SOC + Slow_SOC  # Sum of the two SOC values
                    
                    # Ensure the date format is correct: ensure month has two digits
                    day, month, year = date.split('/')
                    month = month.zfill(2)  # Ensure month has 2 digits
                    formatted_date = f'{day}/{month}/{year}'  # Reconstruct date with correct month format
                    
                    # Append to file_data
                    file_data.append((formatted_date, SOC_value))
                except ValueError:
                    # Handle lines with non-numeric data
                    print(f"Skipping line in {input_file} due to non-numeric data: {line}")

    # If this is the first file, initialize final_data with dates
    if idx == 0:
        final_data = [[date] for date, _ in file_data]

    # Add SOC values for the current suffix to final_data
    for i, (_, soc_value) in enumerate(file_data):
        final_data[i].append(soc_value)

# Write the combined data to the output file
with open(output_file, 'w') as file:
    # Write the header
    header = ['Date'] + file_suffixes
    file.write('\t'.join(header) + '\n')

    # Write each row of data
    for row in final_data:
        file.write('\t'.join(map(str, row)) + '\n')

print(f"Combined data saved to {output_file}")

### 4.2 Organic layer SOC C2

In [ ]:
encodings = ['latin1', 'iso-8859-1', 'utf-8']
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# List of suffixes for the files to process
file_suffixes = ['OL1', 'OL2', 'OL3', 'OL4', 'OL5', 'OL6']

# Define the output file
output_directory = f'{path}reanalysis/'
output_file = f'{output_directory}Organic_SOC_C2.txt'

# Initialize a list to store the final data table
final_data = []

# Process each file
for idx, suffix in enumerate(file_suffixes):
    input_file = f'{path}reanalysis/inca_output_ERA5.{suffix}'
    
    # Try reading the file with different encodings
    for encoding in encodings:
        try:
            with open(input_file, 'r', encoding=encoding) as file:
                lines = file.readlines()
            break  # If reading is successful, break out of the loop
        except UnicodeDecodeError:
            print(f"Failed to decode {input_file} with encoding {encoding}. Trying next encoding...")

    # Initialize variables for processing
    reach_c2_found = False
    file_data = []

    # Process each line
    for line in lines:
        # Check for the start of the relevant data block
        if 'Sub-catchment 2 - 2' in line:
            reach_c2_found = True
            continue  # Skip this line and move to the next

        # Process lines within the relevant block
        if reach_c2_found:
            # Split the line into columns
            columns = line.split()

            # Check if the line has the required number of columns
            if len(columns) >= 7:
                try:
                    # Extract the date and SOC value
                    date = columns[0]  # Date (first column)
                    Fast_SOC = float(columns[2])  # Second column (convert to float)
                    Slow_SOC = float(columns[6])  # Sixth column (convert to float)
                    SOC_value = Fast_SOC + Slow_SOC  # Sum of the two SOC values
                    
                    # Ensure the date format is correct: ensure month has two digits
                    day, month, year = date.split('/')
                    month = month.zfill(2)  # Ensure month has 2 digits
                    formatted_date = f'{day}/{month}/{year}'  # Reconstruct date with correct month format
                    
                    # Append to file_data
                    file_data.append((formatted_date, SOC_value))
                except ValueError:
                    # Handle lines with non-numeric data
                    print(f"Skipping line in {input_file} due to non-numeric data: {line}")

    # If this is the first file, initialize final_data with dates
    if idx == 0:
        final_data = [[date] for date, _ in file_data]

    # Add SOC values for the current suffix to final_data
    for i, (_, soc_value) in enumerate(file_data):
        final_data[i].append(soc_value)

# Write the combined data to the output file
with open(output_file, 'w') as file:
    # Write the header
    header = ['Date'] + file_suffixes
    file.write('\t'.join(header) + '\n')

    # Write each row of data
    for row in final_data:
        file.write('\t'.join(map(str, row)) + '\n')

print(f"Combined data saved to {output_file}")

### 4.3 Mineral layer SOC C1

In [ ]:
encodings = ['latin1', 'iso-8859-1', 'utf-8']
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# List of suffixes for the files to process
file_suffixes = ['ML1', 'ML2', 'ML3', 'ML4', 'ML5', 'ML6']

# Define the output file
output_directory = f'{path}reanalysis/'
output_file = f'{output_directory}Mineral_SOC_C1.txt'

# Initialize a list to store the final data table
final_data = []

# Process each file
for idx, suffix in enumerate(file_suffixes):
    input_file = f'{path}reanalysis/inca_output_ERA5.{suffix}'
    
    # Try reading the file with different encodings
    for encoding in encodings:
        try:
            with open(input_file, 'r', encoding=encoding) as file:
                lines = file.readlines()
            break  # If reading is successful, break out of the loop
        except UnicodeDecodeError:
            print(f"Failed to decode {input_file} with encoding {encoding}. Trying next encoding...")

    # Initialize variables for processing
    reach_c1_found = False
    file_data = []

    # Process each line
    for line in lines:
        # Check for the start of the relevant data block
        if 'Sub-catchment 1 - 1' in line:
            reach_c1_found = True
            continue  # Skip this line and move to the next

        # Break if the end of the relevant block is found
        if 'Sub-catchment 2 - 2' in line:
            break

        # Process lines within the relevant block
        if reach_c1_found:
            # Split the line into columns
            columns = line.split()

            # Check if the line has the required number of columns
            if len(columns) >= 7:
                try:
                    # Extract the date and SOC value
                    date = columns[0]  # Date (first column)
                    Fast_SOC = float(columns[2])  # Second column (convert to float)
                    Slow_SOC = float(columns[6])  # Sixth column (convert to float)
                    SOC_value = Fast_SOC + Slow_SOC  # Sum of the two SOC values
                    
                    # Ensure the date format is correct: ensure month has two digits
                    day, month, year = date.split('/')
                    month = month.zfill(2)  # Ensure month has 2 digits
                    formatted_date = f'{day}/{month}/{year}'  # Reconstruct date with correct month format
                    
                    # Append to file_data
                    file_data.append((formatted_date, SOC_value))
                except ValueError:
                    # Handle lines with non-numeric data
                    print(f"Skipping line in {input_file} due to non-numeric data: {line}")

    # If this is the first file, initialize final_data with dates
    if idx == 0:
        final_data = [[date] for date, _ in file_data]

    # Add SOC values for the current suffix to final_data
    for i, (_, soc_value) in enumerate(file_data):
        final_data[i].append(soc_value)

# Write the combined data to the output file
with open(output_file, 'w') as file:
    # Write the header
    header = ['Date'] + file_suffixes
    file.write('\t'.join(header) + '\n')

    # Write each row of data
    for row in final_data:
        file.write('\t'.join(map(str, row)) + '\n')

print(f"Combined data saved to {output_file}")


### 4.4 Mineral layer SOC C2

In [ ]:
encodings = ['latin1', 'iso-8859-1', 'utf-8']
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# List of suffixes for the files to process
file_suffixes = ['ML1', 'ML2', 'ML3', 'ML4', 'ML5', 'ML6']

# Define the output file
output_directory = f'{path}reanalysis/'
output_file = f'{output_directory}Mineral_SOC_C2.txt'

# Initialize a list to store the final data table
final_data = []

# Process each file
for idx, suffix in enumerate(file_suffixes):
    input_file = f'{path}reanalysis/inca_output_ERA5.{suffix}'
    
    # Try reading the file with different encodings
    for encoding in encodings:
        try:
            with open(input_file, 'r', encoding=encoding) as file:
                lines = file.readlines()
            break  # If reading is successful, break out of the loop
        except UnicodeDecodeError:
            print(f"Failed to decode {input_file} with encoding {encoding}. Trying next encoding...")

    # Initialize variables for processing
    reach_c1_found = False
    file_data = []

    # Process each line
    for line in lines:
        # Check for the start of the relevant data block
        if 'Sub-catchment 2 - 2' in line:
            reach_c1_found = True
            continue  # Skip this line and move to the next

        # Process lines within the relevant block
        if reach_c1_found:
            # Split the line into columns
            columns = line.split()

            # Check if the line has the required number of columns
            if len(columns) >= 7:
                try:
                    # Extract the date and SOC value
                    date = columns[0]  # Date (first column)
                    Fast_SOC = float(columns[2])  # Second column (convert to float)
                    Slow_SOC = float(columns[6])  # Sixth column (convert to float)
                    SOC_value = Fast_SOC + Slow_SOC  # Sum of the two SOC values
                    
                    # Ensure the date format is correct: ensure month has two digits
                    day, month, year = date.split('/')
                    month = month.zfill(2)  # Ensure month has 2 digits
                    formatted_date = f'{day}/{month}/{year}'  # Reconstruct date with correct month format
                    
                    # Append to file_data
                    file_data.append((formatted_date, SOC_value))
                except ValueError:
                    # Handle lines with non-numeric data
                    print(f"Skipping line in {input_file} due to non-numeric data: {line}")

    # If this is the first file, initialize final_data with dates
    if idx == 0:
        final_data = [[date] for date, _ in file_data]

    # Add SOC values for the current suffix to final_data
    for i, (_, soc_value) in enumerate(file_data):
        final_data[i].append(soc_value)

# Write the combined data to the output file
with open(output_file, 'w') as file:
    # Write the header
    header = ['Date'] + file_suffixes
    file.write('\t'.join(header) + '\n')

    # Write each row of data
    for row in final_data:
        file.write('\t'.join(map(str, row)) + '\n')

print(f"Combined data saved to {output_file}")

# 5. Run INCA

### 5.1 Obtaining ouputs for subcatchment C2

In [ ]:
# Define the encodings to try
encodings = ['latin1', 'iso-8859-1', 'utf-8']

path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# Function to get the correct SOC values for a given date
def get_soc_values(lines_soc, target_date):
    for line in lines_soc:
        # Check for matching date
        if line.startswith(target_date):  # Match date in '1/MM/YYYY' format
            return line.strip().split('\t')[1:7]  # Extract columns 1–5
    return ['NaN'] * 5  # Return NaNs if date not found

# Loop through members, years, and months
for member in range(0, 25):  # member from 0 to 24
    for y in range(0, 24):  # year from 0 to 23
        for m in range(0, 12):  # month from 0 to 11

            # Calculate the target date for each iteration
            month = 1 + m
            year = 1993 + y
            start_year = year - 5                     # IMPORTANT: Change warmup period if required!!!!!!!
            target_date = f'1/{month:02d}/{start_year}'  # Ensure the month is two digits

            # Log the date being processed
            print(f"Processing: {target_date}")  # Verify the date is updated correctly
            
            # Update inca.dat files
            shutil.copy(f'{path}/hindcast_inca_files/inca_C1_member{member}_year{y}_month{m}.dat', path + 'seasonal/hindcast_run_model/inca_C1.dat')
            shutil.copy(f'{path}/hindcast_inca_files/inca_C2_member{member}_year{y}_month{m}.dat', path + 'seasonal/hindcast_run_model/inca_C2.dat')

            # Read necessary files
            with open(f'{path}seasonal/hindcast_bc_persist_input/hind_merged__member{member}_year{y}_month{m}.csv', 'r') as file:
                lines_dat = file.readlines()  # Take Steps number
                
            with open(f'{path}reanalysis/Organic_SOC_C1.txt', 'r') as file:
                lines_Org_SOC = file.readlines()
                
            with open(f'{path}reanalysis/Mineral_SOC_C1.txt', 'r') as file:
                lines_Min_SOC = file.readlines()
                
            with open(f'{path}seasonal/hindcast_run_model/inca_par.par', 'r') as file:
                lines = file.readlines()

            # Update Date and Steps in inca_par.par file
            print(f"Updating inca_par.par with date {target_date}")
            lines[15] = f'{target_date}\n'  # Correctly update Date
            lines[16] = lines_dat[0]  # Update Steps

            # Update Organic_SOC and Mineral_SOC values based on the target date
            org_soc_values = get_soc_values(lines_Org_SOC, target_date)
            min_soc_values = get_soc_values(lines_Min_SOC, target_date)

            # Log the SOC values to ensure they are correctly extracted
            # print(f"Organic SOC values for {target_date}: {org_soc_values}")
            # print(f"Mineral SOC values for {target_date}: {min_soc_values}")

            # Update lines 8 and 11 with SOC values
            lines[7] = ' '.join(org_soc_values) + '\n'
            lines[10] = ' '.join(min_soc_values) + '\n'

            # Write updated inca_par.par file
            with open(f'{path}seasonal/hindcast_run_model/inca_par.par', 'w') as file:
                file.writelines(lines)

            print(lines[15])
            print(lines[7])
            print(lines[10])
            print("--------------------------------------------------------------------------------------")

            # readFile = open(f'{path}seasonal/persist_inca/inca_par.par', 'r')    read the updated .par file to check

            # line = readFile.readline()
            # while line:
                # print(line)
                # line = readFile.readline()
           # print("--------------------------------------------------------------------------------------")


       
            # Run INCA simulation
            os.chdir(path + 'seasonal/hindcast_run_model/')
            os.system("inca_c_cmd_beta8.exe -par inca_par.par -dat inca_C1.dat inca_C2.dat -spatial inca_sts.sts")

            # time.sleep(2)   this adds time to run the model and then take the next run.

            # Process INCA output
            for encoding in encodings:
                try:
                    with open(f'{path}seasonal/hindcast_run_model/inca_out.dsd', 'r', encoding=encoding) as file:
                        lines = file.readlines()
                    break
                except UnicodeDecodeError:
                    print(f"Failed to decode with encoding {encoding}. Trying next encoding...")

            # Initialize variables for processing INCA output
            reach_c2_found = False
            data_lines = []

            # Process each line in INCA output
            for line in lines:
                if 'Reach 2 - 2' in line:
                    reach_c2_found = True
                    continue
                
                if reach_c2_found:
                    columns = line.split()
                    if len(columns) >= 4:
                        date = columns[0]
                        flow = columns[1]
                        open_water_doc = columns[4]
                        data_lines.append((date, flow, open_water_doc))

            # Skip the first three lines of data
            data_lines = data_lines[3:]

            # Save processed output
            output_directory = f'{path}seasonal/hindcast_inca_output/'
            output_file = f'{output_directory}inca_output_C2_member{member+1}_year{year}_month{month}.txt'

            with open(output_file, 'w') as file:
                for item in data_lines:
                    file.write(f"{item[0]}\t{item[1]}\t{item[2]}\n")

            print(f"Saved processed output for {target_date} to {output_file}")   


### 5.2 Otaining outputs for subcatchment C1

In [ ]:
# Define the encodings to try
encodings = ['latin1', 'iso-8859-1', 'utf-8']

path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# Function to get the correct SOC values for a given date
def get_soc_values(lines_soc, target_date):
    for line in lines_soc:
        # Check for matching date
        if line.startswith(target_date):  # Match date in '1/MM/YYYY' format
            return line.strip().split('\t')[1:7]  # Extract columns 1–5
    return ['NaN'] * 5  # Return NaNs if date not found

# Loop through members, years, and months
for member in range(0, 25):  # member from 0 to 24
    for y in range(0, 24):  # year from 0 to 23
        for m in range(0, 12):  # month from 0 to 11

            # Calculate the target date for each iteration
            month = 1 + m
            year = 1993 + y
            start_year = year - 5
            target_date = f'1/{month:02d}/{start_year}'  # Ensure the month is two digits

            # Log the date being processed
            print(f"Processing: {target_date}")  # Verify the date is updated correctly
            
            # Update inca.dat files
            shutil.copy(f'{path}seasonal/hindcast_inca_files/inca_C1_member{member}_year{y}_month{m}.dat', path + 'seasonal/hindcast_run_model/inca_C1.dat')
            shutil.copy(f'{path}seasonal/hindcast_inca_files/inca_C2_member{member}_year{y}_month{m}.dat', path + 'seasonal/hindcast_run_model/inca_C2.dat')
        
            # Read necessary files
            with open(f'{path}seasonal/hindcast_bc_persist_input/hind_merged__member{member}_year{y}_month{m}.csv', 'r') as file:
                lines_dat = file.readlines()  # Take Steps number
                
            with open(f'{path}reanalysis/Organic_SOC_C1.txt', 'r') as file:
                lines_Org_SOC = file.readlines()
                
            with open(f'{path}reanalysis/Mineral_SOC_C1.txt', 'r') as file:
                lines_Min_SOC = file.readlines()
                
            with open(f'{path}seasonal/hindcast_run_model/inca_par.par', 'r') as file:
                lines = file.readlines()

            # Update Date and Steps in inca_par.par file
            print(f"Updating inca_par.par with date {target_date}")
            lines[15] = f'{target_date}\n'  # Correctly update Date
            lines[16] = lines_dat[0]  # Update Steps

            # Update Organic_SOC and Mineral_SOC values based on the target date
            org_soc_values = get_soc_values(lines_Org_SOC, target_date)
            min_soc_values = get_soc_values(lines_Min_SOC, target_date)

            # Log the SOC values to ensure they are correctly extracted
            print(f"Organic SOC values for {target_date}: {org_soc_values}")
            print(f"Mineral SOC values for {target_date}: {min_soc_values}")

            # Update lines 8 and 11 with SOC values
            lines[7] = ' '.join(org_soc_values) + '\n'
            lines[10] = ' '.join(min_soc_values) + '\n'

            # Write updated inca_par.par file
            with open(f'{path}seasonal/hindcast_run_model/inca_par.par', 'w') as file:
                file.writelines(lines)

            print(lines[15])
            print(lines[7])
            print(lines[10])
            print("--------------------------------------------------------------------------------------")

            #readFile = open(f'{path}seasonal/persist_inca/inca_par.par', 'r')

            #line = readFile.readline()
            #while line:
                #print(line)
                #line = readFile.readline()
            #print("--------------------------------------------------------------------------------------")


       
            # Run INCA simulation
            os.chdir(path + 'seasonal/hindcast_run_model/')
            os.system("inca_c_cmd_beta8.exe -par inca_par.par -dat inca_C1.dat inca_C2.dat -spatial inca_sts.sts")

            #time.sleep(2)
    
    # Save INCA output files
    #shutil.copy('inca_out.dsd', path + f'hindcast_output_bc/inca_out_member{member}_year{year}_month{month}.txt')
    
    #Process INCA output .dsd files
    # Try opening the file with different encodings if UTF-8 fails
            for encoding in encodings:
                try:
                    with open(f'{path}hindcast_run_model/inca_out.dsd', 'r', encoding=encoding) as file:
                        lines = file.readlines()
                    break  # If reading is successful, break out of the loop
                except UnicodeDecodeError:
                    print(f"Failed to decode with encoding {encoding}. Trying next encoding...")

            # Initialize variables
            reach_c1_found = False
            data_lines = []

            # Process each line
            for line in lines:
                if 'Reach 1 - 1' in line:
                    reach_c1_found = True
                    continue
                
                if 'Reach 2 -2' in line:
                    break
                
                if reach_c1_found:
                     # Split the line into columns
                    columns = line.split()
                    
                    # Check if the line has the required number of columns
                    if len(columns) >= 4:
                    # Extract the Date (First column), Flow (2nd column) and Open water DOC (5th column)
                        date = columns[0]
                        flow = columns[1]
                        open_water_doc = columns[4]
                        data_lines.append((date, flow, open_water_doc))

            # Skip the first three lines
            data_lines = data_lines[3:]

            # Define the output directory
            output_directory = f'{path}hindcast_inca_output/'

            # Save data_lines as a text file
            output_file = f'{output_directory}inca_output_C1_member{member}_year{year}_month{month}.txt'
            with open(output_file, 'w') as file:
                for item in data_lines:
                    file.write(f"{item[0]}\t{item[1]}\t{item[2]}\n")

# 6. Processing INCA output from ERA5 (when necessary)

In [7]:
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path) #always run this line before runing a new test

# Define the encodings to try
encodings = ['latin1', 'iso-8859-1', 'utf-8']

#Process INCA output .dsd files
    # Try opening the file with different encodings if UTF-8 fails
for encoding in encodings:
    try:
        with open(f'{path}reanalysis/inca_out.dsd', 'r', encoding=encoding) as file:  # inca_output_SEAS5_member0_year17_month2_warmup.dsd'
            lines = file.readlines()
        break  # If reading is successful, break out of the loop
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding {encoding}. Trying next encoding...")

# Initialize variables
reach_c2_found = False
data_lines = []

# Process each line
for line in lines:
    if 'Reach 2 - 2' in line:
        
        reach_c2_found = True
        continue
                
    if reach_c2_found:
            # Split the line into columns
        columns = line.split()
                    
        # Check if the line has the required number of columns
        if len(columns) >= 4:
        # Extract the Date (First column), Flow (2nd column) and Open water DOC (5th column)
            date = columns[0]
            flow = columns[1]
            open_water_doc = columns[4]
            data_lines.append((date, flow, open_water_doc))

# Skip the first three lines
data_lines = data_lines[3:]

# Define the output directory
output_directory = f'{path}reanalysis/'

# Save data_lines as a text file
output_file = f'{output_directory}inca_out_C2_ERA5.txt'    #SEAS5_warmup
with open(output_file, 'w') as file:
    for item in data_lines:
        file.write(f"{item[0]}\t{item[1]}\t{item[2]}\n")